In [2]:
%load_ext autoreload
%autoreload 2

In [41]:
import pandas as pd
from transformers import AutoTokenizer

model_name = "Qwen/Qwen3-0.6B-Base"

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

### epoch 1, N_sup = 8

In [10]:
cd ..


/Users/bw/dev/knowledge-sep


In [11]:
df1 = pd.read_json('evals/checkpoint_epoch_1_Nsup8.jsonl', lines=True)

In [12]:
df1

,id,prompt,generated_answer,final_answer,is_match
0,1,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of eggs she eats in ...,16,False
1,2,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of bolts of fabric i...,<think>\nThe total number of bolts of fabric i...,False
2,3,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nThe total cost of the house is $80,00...","65,000",False
3,4,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of meters he runs in...,180,False
4,5,"<|im_start|>user\nEvery day, Wendi feeds each ...",<think>\nThe total number of cups of feed that...,30,False
...,...,...,...,...,...
195,196,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe cost of a pen is $1.20 + $0.30 = ...,24.00,False
196,197,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total amount of carrots is 200 po...,120,True
197,198,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of people in the cit...,16276,False
198,199,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of cards she has is ...,140,False


In [13]:
df1['is_match'].mean()

np.float64(0.195)

In [14]:
print(df1.iloc[7]['prompt'])

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user
Carla is downloading a 200 GB file. Normally she can download 2 GB/minute, but 40% of the way through the download, Windows forces a restart to install updates, which takes 20 minutes. Then Carla has to restart the download from the beginning. How load does it take to download the file?<|im_end|>
<|im_start|>assistant



In [94]:
print(df1['generated_answer'].iloc[7])

<think>
The total time is 200/2 = <<200/2=100>>100 minutes.
The first 100 minutes is 100 minutes.
The second 20 minutes is 20 minutes.
So the total time is 100+20 = <<120>>120 minutes.
</think>

Answer: 120 minutes<|im_end|>
<|endoftext|>The total time to download the 200-GB file is 120 minutes.<|im_end|>
<|endoftext|>In a 200-GB file, Carla can download 200/2 = <<200/2=100>>100 minutes on the first 100 minutes of the file. 40% of the time is 20 minutes of the second 20 minutes of the 200-GB file. 100+20 = <<120>>120 minutes in total.<|im_end|>
<|endoftext|>In a 200-GB file, the total time to be downloaded is 120 minutes.<|im_end|>
<|endoftext|>Answer: 120<|im_end|>
<|endoftext|>Answer: 120<|im_end|>
<|endoftext|>


In [16]:
df1['generated_answer'].apply(lambda ga: ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()).iloc[7]

'<think>\nThe total time is 200/2 = <<200/2=100>>100 minutes.\nThe first 100 minutes is 100 minutes.\nThe second 20 minutes is 20 minutes.\nSo the total time is 100+20 = <<120>>120 minutes.\n</think>\n\nAnswer: 120 minutes'

In [17]:
def process_answer(ga):
    # Cut off parts after EOS token
    ga = ga.split(tokenizer.special_tokens_map['eos_token'])[0].strip()

    # Extract the final answer after the last "Answer:" marker
    if "Answer:" in ga:
        ga = ga.split("Answer:")[-1].strip()
        ga = ga.split(' ')[0].strip()
    return ga

df1['new_final_answer'] = df1['generated_answer'].apply(process_answer)

In [18]:
df1.iloc[7][['final_answer', 'new_final_answer']]

final_answer        120
new_final_answer    120
Name: 7, dtype: object

In [19]:
from data import get_generation_dataloader, get_dataloader

In [20]:
eval_loader = get_generation_dataloader(
    tokenizer, max_length=32768, batch_size=2,
    seed=42, train=False, num_samples=None
)

Loading dataset for generation evaluation...


In [21]:
batch = None
for example in eval_loader:
    batch = example
    break

In [22]:
batch['prompts']

["<|im_start|>user\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|im_end|>\n<|im_start|>assistant\n",
 '<|im_start|>user\nA robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|im_end|>\n<|im_start|>assistant\n']

In [23]:
prompt_texts = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=False)

In [24]:
prompt_texts

["<|im_start|>user\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?<|im_end|>\n<|im_start|>assistant\n",
 '<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>user\nA robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|im_end|>\n<|im_start|>assist

In [66]:
example = {'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'}

In [69]:
messages = [
    {'role': 'user', 'content': example['question']}
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

In [70]:
print(prompt)

<|im_start|>user
A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?<|im_end|>
<|im_start|>assistant



In [56]:
trn_loader = get_dataloader(
    tokenizer, 32768, 2,
    seed=42, train=True, num_samples=None
)

Loading dataset...


In [57]:
batch = None
for example in trn_loader:
    batch = example
    break

In [59]:
batch['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [61]:
decoded = tokenizer.batch_decode(batch['input_ids'])
decoded

['<|im_start|>user\nMimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?<|im_end|>\n<|im_start|>assistant\n<think>\nMimi has 2 x 12 = <<2*12=24>>24 sea shells.\nKyle has 24 x 2 = <<24*2=48>>48 sea shells.\nLeigh has 48 / 3 = <<48/3=16>>16 sea shells.\n</think>\n\nAnswer: 16<|im_end|>\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 "<|im_start|>user\nFrankie's parents let him have many pets. He has six more snakes than he has cats. He has one less parrot than cats. Six of his pets have four legs. He has 2 dogs. How many pets does he have in total?<|im_end|>\n<|im_start|>assistant\n<think>\nHe has 6 - 2 = <<6-2

In [63]:
print(decoded[1])

<|im_start|>user
Frankie's parents let him have many pets. He has six more snakes than he has cats. He has one less parrot than cats. Six of his pets have four legs. He has 2 dogs. How many pets does he have in total?<|im_end|>
<|im_start|>assistant
<think>
He has 6 - 2 = <<6-2=4>>4 cats.
He has 4 - 1 = <<4-1=3>>3 parrots.
He has 4 + 6 = <<4+6=10>>10 snakes.
He has a total of 2 + 4 + 3 + 10 = <<2+4+3+10=19>>19 pets.
</think>

Answer: 19<|im_end|>



In [11]:
ground_truths = []
for i, example in enumerate(eval_loader):
    if i >= 200:
        break
    ground_truths.append({
        'id': i,
        'ground_truth': example['ground_truths'][0],
    })

In [12]:
gt_df = pd.DataFrame(ground_truths)

In [23]:
df1['ground_truth'] = [r['ground_truth'] for r in ground_truths]

In [26]:
def new_is_match(row):
    gt = row['ground_truth']
    new_ans = row['new_final_answer']
    try:
        new_is_match = (float(gt) == float(new_ans))
    except:
        new_is_match = (gt == new_ans)
    return new_is_match

In [29]:
df1['new_is_match'] = df1.apply(new_is_match, axis=1)

In [31]:
df1['new_is_match'].mean()

np.float64(0.195)

In [39]:
df1.head()

,id,prompt,generated_answer,final_answer,is_match,new_final_answer,ground_truth,new_is_match
0,1,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of eggs she eats in ...,16,False,16,18,False
1,2,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of bolts of fabric i...,<think>\nThe total number of bolts of fabric i...,False,<think>\nThe total number of bolts of fabric i...,3,False
2,3,<|endoftext|><|endoftext|><|endoftext|><|endof...,"<think>\nThe total cost of the house is $80,00...","65,000",False,"65,000",70000,False
3,4,<|endoftext|><|endoftext|><|endoftext|><|endof...,<think>\nThe total number of meters he runs in...,180,False,180,540,False
4,5,"<|im_start|>user\nEvery day, Wendi feeds each ...",<think>\nThe total number of cups of feed that...,30,False,30,20,False


In [45]:
row = df1.iloc[3]
print(row['prompt'])

<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|im_start|>

In [43]:
print(row['generated_answer'])

<think>
The total cost of the house is $80,000 + $50,000 = $<<130,000>>130,000.
The value of the house is 150% of the total cost, so 150/100 * $130,000 = $<<195,000>>195,000.
The profit is the total cost of the house and the value of the house, so $195,000 - $130,000 = $<<65,000>>65,000.
</think>

Answer: 65,000<|im_end|>
<|endoftext|>In a 150% value increase, the total cost of the house is $80,000 + $50,000 = $130,000.
The value of the house is 150% of the total cost, so 150/100 * $130,000 = $195,000.
The profit is the total cost of the house


### epoch 1, N_sup = 1

In [32]:
df2 = pd.read_json('evals/checkpoint_epoch_1_Nsup1.jsonl', lines=True)

In [33]:
df2['is_match'].mean()

np.float64(0.2)

In [34]:
df2['new_final_answer'] = df2['generated_answer'].apply(process_answer)

In [35]:
df2['ground_truth'] = [r['ground_truth'] for r in ground_truths]

In [36]:
df2['new_is_match'] = df2.apply(new_is_match, axis=1)

In [38]:
df2['new_is_match'].mean()

np.float64(0.215)

### Base model

In [72]:
df3 = pd.read_json('evals/checkpoint_epoch_1_only_base.jsonl', lines=True)

In [97]:
print(df3.iloc[17]['generated_answer'])

<think>
Okay, let's see. Jill's annual salary is calculated by multiplying her hourly wage by the number of hours she works each week, and then multiplying that by the number of weeks in a year. But wait, there's also the coach part. So first, I need to break down her earnings into teacher hours and coach hours separately.

Starting with the teacher part. She gets paid $20 per hour as a teacher. She works 35 hours a week. So, teacher earnings per week would be 35 hours multiplied by $20. Let me calculate that. 35 times 20... Hmm, 30 times 20 is 600, plus 5 times 20 is 100, so total is 700. So that's $700 per week for teaching.

Now for the cheerleading coach part. She gets $30 to be a cheerleader coach. Wait, is that the same as the teacher's wage? Or is that a separate payment? The problem says she gets paid $20 per hour for teaching and $30 to be a cheerleading coach. So that's two separate components. So she works 15 hours a week as a coach. So coach earnings per week would be 15 ho